Load the Data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
import glob
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras import layers, models


Mounted at /content/drive


Mount Drive & set paths

In [2]:
DATA_DIR = "/content/drive/MyDrive/Syncrones/Syncrones_A"  # ONLY අ images

IMG_SIZE   = (64, 64)
BATCH_SIZE = 32
SEED       = 42

Load all images as a list of paths

In [3]:
image_paths = []
for ext in ("*.png", "*.jpg", "*.jpeg", "*.PNG", "*.JPG", "*.JPEG"):
    image_paths.extend(glob.glob(os.path.join(DATA_DIR, ext)))

image_paths = sorted(image_paths)
print("Total අ images found:", len(image_paths))

if len(image_paths) == 0:
    raise ValueError("No images found in DATA_DIR. Check path and file types.")

Total අ images found: 60


 Split into train / val

In [4]:
num_total  = len(image_paths)
train_size = int(0.8 * num_total)

train_paths = np.array(image_paths[:train_size])
val_paths   = np.array(image_paths[train_size:])

print("Train images:", len(train_paths))
print("Val images:  ", len(val_paths))


Train images: 48
Val images:   12


tf.data pipeline (images only)

In [5]:
def load_and_preprocess(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_image(image, channels=1, expand_animations=False)
    image = tf.image.resize(image, IMG_SIZE)
    image = tf.cast(image, tf.float32) / 255.0
    return image

train_ds = tf.data.Dataset.from_tensor_slices(train_paths)
train_ds = (
    train_ds
    .map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    .shuffle(1000, seed=SEED)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)

val_ds = tf.data.Dataset.from_tensor_slices(val_paths)
val_ds = (
    val_ds
    .map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)

# For autoencoder, targets = inputs -> (x, x)
train_ds_ae = train_ds.map(lambda x: (x, x))
val_ds_ae   = val_ds.map(lambda x: (x, x))

# Quick sanity check
for batch_x, batch_y in train_ds_ae.take(1):
    print("Batch x shape:", batch_x.shape)
    print("Batch y shape:", batch_y.shape)

Batch x shape: (32, 64, 64, 1)
Batch y shape: (32, 64, 64, 1)


Autoencoder model

In [6]:
input_shape = (IMG_SIZE[0], IMG_SIZE[1], 1)

encoder_input = layers.Input(shape=input_shape)

x = layers.Conv2D(32, (3, 3), padding='same')(encoder_input)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)

x = layers.Conv2D(64, (3, 3), padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)

x = layers.Conv2D(128, (3, 3), padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
encoded = layers.MaxPooling2D((2, 2), padding='same', name="encoded")(x)

# Decoder
x = layers.Conv2DTranspose(128, (3, 3), strides=2, padding='same')(encoded)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)

x = layers.Conv2DTranspose(64, (3, 3), strides=2, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)

x = layers.Conv2DTranspose(32, (3, 3), strides=2, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)

decoded = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = models.Model(encoder_input, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

autoencoder.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 64, 64, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 64, 64, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 64, 64, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 32, 32, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 32, 32, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 16, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 16, 16, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoded (MaxPooling2D)          │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose                │ (None, 16, 16, 128)    │       147,584 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 16, 16, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_1              │ (None, 32, 32, 64)     │        73,792 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 32, 32, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_2              │ (None, 64, 64, 32)     │        18,464 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 64, 64, 32)     │           12

 Total params: 334,593 (1.28 MB)

 Trainable params: 333,697 (1.27 MB)

 Non-trainable params: 896 (3.50 KB)

Train

In [7]:
EPOCHS = 50

history = autoencoder.fit(
    train_ds_ae,
    validation_data=val_ds_ae,
    epochs=EPOCHS
)

Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - loss: 0.0621 - val_loss: 0.0723
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 634ms/step - loss: 0.0268 - val_loss: 0.0710
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 652ms/step - loss: 0.0139 - val_loss: 0.0693
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - loss: 0.0105 - val_loss: 0.0673
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - loss: 0.0103 - val_loss: 0.0653
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 620ms/step - loss: 0.0100 - val_loss: 0.0633
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - loss: 0.0101 - val_loss: 0.0615
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 684ms/step - loss: 0.0092 - val_loss: 0.0600
Epoch 9/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 668ms/step - loss: 0.0084 - val_loss: 0.0588
Epoch 10/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - loss: 0.0076 - val_loss: 0.0578
Epoch 11/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 655ms/step - loss: 0.0073 - val_loss: 0.0570
Epoch 12/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 692ms/step - loss: 0.0069 - val_loss: 0.0560
Epo

Compute reconstruction-error threshold

In [8]:
recon_errors = []

for batch in train_ds:
    reconstructed = autoencoder.predict(batch)
    batch = batch.numpy()
    batch_errors = np.mean((batch - reconstructed)**2, axis=(1, 2, 3))
    recon_errors.extend(batch_errors)

recon_errors = np.array(recon_errors)
print("Train errors: min:", recon_errors.min(), "max:", recon_errors.max())
print("Mean:", recon_errors.mean(), "Std:", recon_errors.std())

# Use a percentile-based threshold to make it stricter.
# e.g. 95th percentile: 95% of true 'අ' are below this.
threshold = np.percentile(recon_errors, 95)
print("Reconstruction error threshold (95th percentile):", threshold)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 385ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 316ms/step
Train errors: min: 0.007881753 max: 0.047729917
Mean: 0.023140242 Std: 0.0095750345
Reconstruction error threshold (95th percentile): 0.040920477


Helper to preprocess a single image

In [9]:
def preprocess_single_image(image_path):
    img = Image.open(image_path).convert("L")
    img = img.resize(IMG_SIZE)
    img = np.array(img).astype("float32") / 255.0
    img = np.expand_dims(img, axis=(0, -1))  # (1, H, W, 1)
    return img

def is_A(image_path, threshold=threshold):
    img = preprocess_single_image(image_path)
    reconstructed = autoencoder.predict(img)
    error = np.mean((img - reconstructed)**2)
    print("Reconstruction error:", error)

    if error <= threshold:
        return "Looks like අ (similar to training images)"
    else:
        return "Not similar to trained අ"

Check how many training images are classified as අ

In [10]:
inside = np.sum(recon_errors <= threshold)
outside = np.sum(recon_errors > threshold)

print(f"Training images classified as අ: {inside}/{len(recon_errors)}")
print(f"Training images rejected:       {outside}/{len(recon_errors)}")

Training images classified as අ: 45/48
Training images rejected:       3/48


In [12]:
test_img = "/content/drive/MyDrive/Syncrones/testing/test_f.png"  # change path
result = is_A(test_img)
print("Result:", result)


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Syncrones/testing/test_f.png'